In [6]:
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
# pylint: enable=line-too-long

import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

# Drug design

In [7]:
# Case study: H2 molecule
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.735))]
basis = 'sto3g'
multiplicity = 1
charge = 0
molecule = Molecule(geometry, basis, multiplicity, charge)

# Driver
driver = ElectronicStructureMoleculeDriver(molecule, driver_type=ElectronicStructureDriverType.PYSCF)
properties = driver.run()
num_particles = (properties
                    .get_property("ParticleNumber")
                    .num_particles)
num_spin_orbitals = int(properties
                        .get_property("ParticleNumber")
                        .num_spin_orbitals)

# Define Problem, Use freeze core approximation, remove orbitals.
problem = ElectronicStructureProblem(
    driver,
    [FreezeCoreTransformer(freeze_core=True,
                            remove_orbitals=[-3,-2])])

second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
num_spin_orbitals = problem.num_spin_orbitals
num_particles = problem.num_particles

mapper = ParityMapper()  # Set Mapper
hamiltonian = second_q_ops[0]  # Set Hamiltonian
# Do two qubit reduction
converter = QubitConverter(mapper,two_qubit_reduction=True)
reducer = TwoQubitReduction(num_particles)
qubit_op = converter.convert(hamiltonian)
qubit_op = reducer.convert(qubit_op)

def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

MissingOptionalLibraryError: "The 'pyscf' library is required to use 'PySCFDriver'.  See https://pyscf.org/install.html."

In [2]:
# oxalic acid geometry
#       O
#       |
#       C - C - O - H
#       |   
#       O - H
# Bond lengths in Angstroms
# C-O bond length is 1.43 Angstroms 
# C-C bond length is 1.45 Angstroms
# O-H bond length is 0.96 Angstroms

# set the Molecule for Oxalic Acid
geometry = [('C', (0., 0., 0.)),
            ('O', (0., 0., 1.43)),
            ('O', (0., 0., -1.43)),
            ('C', (1.45, 0., 0.)),
            ('O', (2.48, 0., 0.)),
            ('H', (3.44, 0., 0.)),
            ('H', (0., 0.96, -1.43))]

basis = 'sto-3g'
multiplicity = 1
charge = 0

# set the Molecule for Oxalic Acid
molecule = Molecule(geometry, basis, multiplicity, charge)


In [3]:
molecule